In [ ]:
import numpy as np
import pandas as pd 
from opencage.geocoder import OpenCageGeocode
from pprint import pprint
from ipywidgets import HTML
from ipyleaflet import Popup, Map, Marker, LayersControl
from IPython import display

class desafioBiodiversidade():
    def __init__(self,filePath):
        try:
            f = open(filePath,"r")
            self.originalData = f.readlines()
            self.data = [l.rstrip().split(";") for l in self.originalData]
            f.close()

            self.AmeacaList = []
            self.plotMap = []
            self.dataAux = []
            self.dataT = []            
            
        except:
            print("Error: cannot open file.")
            data = None  
    
    #1
    #Faça um método que retorna a média de dados faltantes por coluna
    def retornaMediaDadosFaltantesPorColuna(self):
        data = self.data
        nlines = len(data)-1
        
        # lista de tamanho igual ao numero de colunas
        nblank = [0]*len(data[0])
        
        # cada posicao da lista armazena o contador de linhas faltantes da coluna correspondente
        for line in data:
            for ind,column in enumerate(line):
                if column == "Sem Informações" or column == "":
                    nblank[ind]+=1
        
        # calcula a media dos dados faltantes de cada coluna
        nblank = [np.round((x/nlines)*100,2) for x in nblank]
        
        # passa a lista para um dicionario
        # para impressao utilizando DataFrame do Pandas
        dict = {'Titulo': data[0], 'Value': nblank}
        df = pd.DataFrame(dict)
        print(df.to_string(index=False))
        
        
    #2
    #Para cada item identifique até qual nível taxônomico a ocorrência foi identificada.
    def retornaNivelTaxonomicoCadaOcorrencia(self):
        lista = self.data

        filo = []
        for i in range(len(lista[0])):
            
            if lista[0][i] == 'Especie':
              
                for k in range(1,len(lista)):
                    
                    if lista[k][i] != 'Sem Informações':
                        filo.append('{} - Especie'.format(k))
                    
                    elif lista[k][i-1] != 'Sem Informações':
                        filo.append('{} - Genero'.format(k))
                        
                    elif lista[k][i-2] != 'Sem Informações':
                        filo.append('{} - Familia'.format(k))
                        
                    elif lista[k][i-3] != 'Sem Informações':
                        filo.append('{} - Ordem'.format(k))
                    
                    elif lista[k][i-4] != 'Sem Informações':
                        filo.append('{} - Classe'.format(k))
                        
                    elif lista[k][i-5] != 'Sem Informações':
                        filo.append('{} - Filo'.format(k))
                        
                    else:
                        filo.append('{} - Reino'.format(k))
     
        return filo  
    
    #3
    #Monte filtros de ocorrências por estados, nome de espécie (nome exato ou parte do nome) e 
    #categoria de ameaça, e outros filtros que julgar relevante.
    def retonaSeOcorrenciaExiste(self, filterAmeaca):

        # Lê cada coluna por vez        
        for j in range(len(self.data[0])):        
            for i in range(len(self.data)-1):
                self.dataAux.append(self.data[i][j])
                # Cria uma nova "matriz" transposta com os dados
            self.dataT.append(self.dataAux)
            self.dataAux = []
        
        
        ##### Filtros de busca
        # Filtro de ameaça
        for i in range(len(self.dataT)):
            if self.dataT[i][0] == "Estado de conservacao":
                for j in range(len(self.dataT[i])):
                    if self.dataT[i][j] == filterAmeaca :
                        self.plotMap.append([self.dataT[i+7][j],self.dataT[i+8][j], self.dataT[i+5][j]])
                        self.AmeacaList.append(["Amostra {}: {}: ".format(j, self.dataT[i-12][j]), filterAmeaca])
                    
        return self.AmeacaList
        
    #4
    #Crie uma funcionalidade para avaliar se a informação de longitude e latitude 
    #corresponde a informação presente na localização
    def metodoPlotMap(self):
        vectorInformacoes = self.plotMap
        key = '0573d875005f45bebe452ff81c26b684'
        geocoder = OpenCageGeocode(key)

        i = 0
        center = (0.0, 0.0)
        itemMapa = Map(center = center, zoom = 3, close_popup_on_click = False)
        itemMapa.add_control(LayersControl())
        for item in vectorInformacoes:
            #if i <= 15:
                center = (item[0], item[1])
                marker = Marker(location = center, draggable = False)
                teste = HTML()
                teste.value = "<b>" + item[2] + "</b>"
                popup = Popup(
                    location = center,
                    child = teste,
                    close_button = True,
                    auto_close = False,
                    close_on_escape_key = False
                )
                itemMapa.add_layer(marker)
                itemMapa.add_layer(popup)
            #i+=1
        print(itemMapa)
    
filePath1 = "portalbio_export_16-10-2019-14-39-54.csv"
filePath2 = "portalbio_export_17-10-2019-13-06-22.csv"
filePath3 = "portalbio_export_17-10-2019-13-15-00.csv"
        

In [ ]:
#PARTE GABRIELA
biodiversidade = desafioBiodiversidade(filePath1)

In [ ]:
biodiversidade.retornaMediaDadosFaltantesPorColuna()
#PARTE GABRIELA

In [ ]:
#PARTE BRUNA
print(biodiversidade.retornaNivelTaxonomicoCadaOcorrencia())
#PARTE BRUNA

In [ ]:
#PARTE RAFAEL
print(biodiversidade.retonaSeOcorrenciaExiste("Espécie Ameaçada"))
#PARTE RAFAEL

In [ ]:
#PARTE RAFAEL + DANIEL
map = biodiversidade.metodoPlotMap()
#PARTE RAFAEL + DANIEL